# group60

### 目標
1. ユーザーが自分のデータを入力できるアプリケーションを作成する。
2. まずはサンプル食品の数字がちゃんと表示できるものを作成する

## 変更箇所　①食塩を省く　②def get_weight()修正　③入力値の正規化これ　④4bit→5bit～

In [381]:
# ライブラリのインポート
!pip install dimod dwave-neal
import numpy as np
from neal import SimulatedAnnealingSampler as SASampler
from dimod import BinaryQuadraticModel, ExactSolver  # QUBOを扱うためのライブラリ

### 1. 問題設定

In [382]:
# 仮の原材料数
# num_ingredients = 4
num_ingredients = 3


In [383]:
# 変数定義　75Gきなこ飴ちゅん太とピーコ
# https://www.yaokin.com/products_search/nostalgic-snack/item_M38015
# 原材料   きなこ（大豆を含む）（国内製造）、水飴、砂糖
# 75 gあたりの成分情報
# カロリー  299 Kcal
# たんぱく質 12.00 g
# 脂質  7.40 g
# 炭水化物  46.40 g
# 食塩相当量 0.000 g

# 元データの重量
data_weight = 75.0 # きなこ飴75gあたりの成分情報 (修正箇所：追加)

target_weight = 75  # 食品の重量

# ターゲット栄養成分値（きなこ飴75gあたり）の定義 (修正箇所：追加)
# これらの値はユーザーが画像と以前のプロンプトで提示した75gあたりの絶対値です。
target_calories_75g = 299.0
target_protein_75g = 12.00
target_fat_75g = 7.40
target_carbs_75g = 46.40
target_salt_75g = 0.000 # コスト関数では不使用

# コスト関数で使用するため、ターゲット値を「製品1gあたり」に変換 (修正箇所：追加)
# 計算栄養成分は (原材料1gあたり) * 比率 の和となり、これは製品1gあたりの栄養成分に相当すると解釈します。
#target_calories = target_calories_75g / data_weight
#target_protein = target_protein_75g / data_weight
#target_fat = target_fat_75g / data_weight
#target_carbs = target_carbs_75g / data_weight
# target_salt = target_salt_75g / data_weight # 使用しない
target_calories_g = target_calories_75g / data_weight
target_protein_g = target_protein_75g / data_weight
target_fat_g = target_fat_75g / data_weight
target_carbs_g = target_carbs_75g / data_weight

target_calories = 1 #* 正規化値追加2025/04/29
target_protein = 1 #* 正規化値追加2025/04/29
target_fat = 1 #* 正規化値追加2025/04/29
target_carbs = 1 #* 正規化値追加2025/04/29

In [384]:
print(target_calories_g)
print(target_protein_g)
print(target_fat_g)
print(target_carbs_g)


3.986666666666667
0.16
0.09866666666666667
0.6186666666666667


In [385]:
# 各原材料の標準栄養成分（カロリー、タンパク質、脂質、炭水化物）
# ingredient:成分、nutrition：栄養
# 各原材料の標準栄養成分（1gあたりの値に変換） (修正箇所：追加)
# ご提示いただいた表の100gあたりの値を100で割ります。
ingredient_keys = ['ingredient1', 'ingredient2', 'ingredient3'] # 修正箇所：追加 (キーリスト化)
#ingredient_nutrition_per_g = {
#    'ingredient1': {'calories': 451.0 / 100.0, 'protein': 36.7 / 100.0, 'fat': 25.7 / 100.0, 'carbs': 28.5 / 100.0, 'salt': 0.0, 'name': 'きなこ（大豆を含む）（国内製造）'},
#    'ingredient2': {'calories': 391.0 / 100.0, 'protein': 0.0 / 100.0, 'fat': 0.0 / 100.0, 'carbs': 99.3 / 100.0, 'salt': 0.0, 'name': '砂糖'},
#    'ingredient3': {'calories': 210.0 / 100.0, 'protein': 0.0 / 100.0, 'fat': 0.0 / 100.0, 'carbs': 69.9 / 100.0, 'salt': 0.0, 'name': '水飴'},
#}

ingredient_nutrition_per_g = {
    'ingredient1': {'calories': 451.0 / 100 / target_calories_g, 'protein': 36.7 / 100 / target_protein_g, 'fat': 25.7 / 100 / target_fat_g, 'carbs': 28.5 / 100 / target_carbs_g, 'salt': 0.0, 'name': 'きなこ（大豆を含む）（国内製造）'},
    'ingredient2': {'calories': 391.0 / 100 / target_calories_g, 'protein': 0.0 / 100 / target_protein_g, 'fat': 0.0 / 100 / target_fat_g, 'carbs': 99.3 / 100 / target_carbs_g, 'salt': 0.0, 'name': '砂糖'},
    'ingredient3': {'calories': 210.0 / 100 / target_calories_g, 'protein': 0.0 / 100 / target_protein_g, 'fat': 0.0 / 100 / target_fat_g, 'carbs': 69.9 / 100 / target_carbs_g, 'salt': 0.0, 'name': '水飴'},
} #* 正規化値追加2025/04/29

In [386]:
ingredient_nutrition_per_g

{'ingredient1': {'calories': 1.1312709030100334,
  'protein': 2.29375,
  'fat': 2.60472972972973,
  'carbs': 0.4606681034482758,
  'salt': 0.0,
  'name': 'きなこ（大豆を含む）（国内製造）'},
 'ingredient2': {'calories': 0.9807692307692307,
  'protein': 0.0,
  'fat': 0.0,
  'carbs': 1.6050646551724137,
  'salt': 0.0,
  'name': '砂糖'},
 'ingredient3': {'calories': 0.5267558528428093,
  'protein': 0.0,
  'fat': 0.0,
  'carbs': 1.1298491379310345,
  'salt': 0.0,
  'name': '水飴'}}

In [387]:
# コスト関数の係数（仮の値）
A = 1.0 # カロリー誤差
B = 1.0 # タンパク質誤差
C = 1.0 # 脂質誤差
D = 1.0 # 炭水化物誤差
E = 0.5 # 原材料順序制約（このスクリプトでは不使用）
F = 100.0 # 合計100%制約（強めに設定） (修正箇所：値を変更)
print("--- Block 1: Setup and Data Loaded ---")
print(f"Target nutritional values per gram (derived from {data_weight}g total):") # 修正箇所：表示内容変更
print(f"  Calories: {target_calories:.4f}")
print(f"  Protein:  {target_protein:.4f}")
print(f"  Fat:      {target_fat:.4f}")
print(f"  Carbs:    {target_carbs:.4f}")

print("\nIngredient nutritional values per gram:") # 修正箇所：表示内容変更
for key, data in ingredient_nutrition_per_g.items(): # 修正箇所：辞書名を変更
    print(f"  {data['name']}: Cal={data['calories']:.4f}, Prot={data['protein']:.4f}, Fat={data['fat']:.4f}, Carb={data['carbs']:.4f}")


--- Block 1: Setup and Data Loaded ---
Target nutritional values per gram (derived from 75.0g total):
  Calories: 1.0000
  Protein:  1.0000
  Fat:      1.0000
  Carbs:    1.0000

Ingredient nutritional values per gram:
  きなこ（大豆を含む）（国内製造）: Cal=1.1313, Prot=2.2938, Fat=2.6047, Carb=0.4607
  砂糖: Cal=0.9808, Prot=0.0000, Fat=0.0000, Carb=1.6051
  水飴: Cal=0.5268, Prot=0.0000, Fat=0.0000, Carb=1.1298


### 2. 二値変数の設定

## コード解説
このコードは、各原材料を4ビットの二値変数で表現するための準備をしています。以下に、各部分を詳しく解説します。

1. num_bits_per_ingredient = 4

これは、「各原材料に対して4ビットの二値変数を割り当てる」という設定です。この変数は、例えば原材料の種類を表現したり、それぞれの値を量子化して表現する際に使われます。

2. bqm = BinaryQuadraticModel({}, {}, offset=0.0, vartype='BINARY')

これはD-Waveのdimodライブラリで使用される「二次形式のバイナリ変数最適化モデル（BQM）」を初期化しています。

引数の説明:

　｛｝： 線形項を表す辞書（初期値として空）。

　｛｝：二次項を表す辞書（初期値として空）。

　offset=0.0 :  定数項。初期値はゼロ。

　vartype='BINARY' : バイナリ変数（0または1）として扱う設定。

BQMは、量子アニーリングやイジングマシンで解くための基本的な数式モデルです。

# bit数の定義

In [388]:
# 各原材料を4ビットの二値変数で表現
num_bits_per_ingredient = 16
bqm = BinaryQuadraticModel({}, {}, offset=0.0, vartype='BINARY')
q = {}  # q[ingredient_index][bit_index] でアクセス（成分、bit位置でアクセス）

In [389]:
q

{}

## コード解説
このコードは、各原材料（ingredient）に対して、指定されたビット数（num_ingredients）だけ二値変数を生成し、BQM（Binary Quadratic Model）に追加するものです。以下に詳細を解説します：

1. ループ構造

for i in range(num_ingredients):

　num_ingredients:  原材料の数を指定する変数（例えばカロリー、タンパク質、脂質、炭水化物で4つ）。


2. 各原材料に対応する二値変数のリストを作成
q[i] = [
    f'q_{i}_{j}' for j in range(num_bits_per_ingredient)
]

目的: 各原材料について、指定されたビット数（num_ingredients）だけ変数を作成します。
構造：
　 f'q_{i}_{j}': 文字列フォーマットを使用して変数名を作成しています。例: q_0_0,q_0_1,q_0_2,q_0_3 （0番目の原材料の4ビット）。

リスト内包表記で効率的に変数を生成。

3. 変数をBQMに追加

for j in range(num_bits_per_ingredient):
    bqm.add_variable(q[i][j])
目的: 作成した各ビットの変数をBQMモデルに追加。

方法：
　bqm.add_variable(): 引数として変数名を渡し、モデルに追加します。
これにより、BQMは後で最適化の対象となる二値変数を認識。


コードの全体的な動作

原材料ごとに指定されたビット数の二値変数を生成します。

各変数はBQMに追加されるため、最適化問題で使用可能になります。

適用例

例えば、以下のような応用があります：

量子アニーリング: 原材料の組み合わせや割合を最適化。

リソース配分: 各原材料のコストや効率を分析。

ポートフォリオ選択: 栄養素のバランスを評価。


In [390]:
# ingredientの数だけループする（４回：カロリー、タンパク質、脂質、炭水化物） # コメントが正確ではない (原材料の数だけループ)
for i in range(num_ingredients):
 # 各ingredientごとに、指定したビット数だけ変数を追加する
   q[i] = [
       f'q_{i}_{j}' for j in range(num_bits_per_ingredient) # 変数名を作成する
     ]
   for j in range(num_bits_per_ingredient):
    bqm.add_variable(q[i][j]) # 変数を追加

In [391]:
bqm

BinaryQuadraticModel({'q_0_0': 0.0, 'q_0_1': 0.0, 'q_0_2': 0.0, 'q_0_3': 0.0, 'q_0_4': 0.0, 'q_0_5': 0.0, 'q_0_6': 0.0, 'q_0_7': 0.0, 'q_0_8': 0.0, 'q_0_9': 0.0, 'q_0_10': 0.0, 'q_0_11': 0.0, 'q_0_12': 0.0, 'q_0_13': 0.0, 'q_0_14': 0.0, 'q_0_15': 0.0, 'q_1_0': 0.0, 'q_1_1': 0.0, 'q_1_2': 0.0, 'q_1_3': 0.0, 'q_1_4': 0.0, 'q_1_5': 0.0, 'q_1_6': 0.0, 'q_1_7': 0.0, 'q_1_8': 0.0, 'q_1_9': 0.0, 'q_1_10': 0.0, 'q_1_11': 0.0, 'q_1_12': 0.0, 'q_1_13': 0.0, 'q_1_14': 0.0, 'q_1_15': 0.0, 'q_2_0': 0.0, 'q_2_1': 0.0, 'q_2_2': 0.0, 'q_2_3': 0.0, 'q_2_4': 0.0, 'q_2_5': 0.0, 'q_2_6': 0.0, 'q_2_7': 0.0, 'q_2_8': 0.0, 'q_2_9': 0.0, 'q_2_10': 0.0, 'q_2_11': 0.0, 'q_2_12': 0.0, 'q_2_13': 0.0, 'q_2_14': 0.0, 'q_2_15': 0.0}, {}, 0.0, 'BINARY')

In [392]:
print(q)
print(q[0])
print(q[0][0])
print(q[0][0][0])
print(q[0][0][0][0])


{0: ['q_0_0', 'q_0_1', 'q_0_2', 'q_0_3', 'q_0_4', 'q_0_5', 'q_0_6', 'q_0_7', 'q_0_8', 'q_0_9', 'q_0_10', 'q_0_11', 'q_0_12', 'q_0_13', 'q_0_14', 'q_0_15'], 1: ['q_1_0', 'q_1_1', 'q_1_2', 'q_1_3', 'q_1_4', 'q_1_5', 'q_1_6', 'q_1_7', 'q_1_8', 'q_1_9', 'q_1_10', 'q_1_11', 'q_1_12', 'q_1_13', 'q_1_14', 'q_1_15'], 2: ['q_2_0', 'q_2_1', 'q_2_2', 'q_2_3', 'q_2_4', 'q_2_5', 'q_2_6', 'q_2_7', 'q_2_8', 'q_2_9', 'q_2_10', 'q_2_11', 'q_2_12', 'q_2_13', 'q_2_14', 'q_2_15']}
['q_0_0', 'q_0_1', 'q_0_2', 'q_0_3', 'q_0_4', 'q_0_5', 'q_0_6', 'q_0_7', 'q_0_8', 'q_0_9', 'q_0_10', 'q_0_11', 'q_0_12', 'q_0_13', 'q_0_14', 'q_0_15']
q_0_0
q
q


### 3. 配合比率 x_i の計算（QUBO形式に組み込む）

## 前提
 - num_ingredients: 3種類の「成分」。
 - num_bits_per_ingredient: 各成分が持つビット数（4ビット）。
 - F: 定数。モデル全体のスケーリング係数として機能。
 - weight: 成分やビットに対する重み。
 - q[i][j]: ビットを表す変数（量子ビットのインデックスと思われます）。

## 各コード部分の解説
###1.自己ループ（二乗項）

　bqm.add_variable(q[i][j], F * weight_i_j * weight_i_j)

　ここでは「自己ループ」、つまり同じ変数の二乗項の係数を設定しています。

　数学的には (F \cdot \text{weight}i][j]^2) を意味します。

　これにより、各ビットが自分自身にどれだけ「影響を与えるか」をモデルに反映します。

###2.同じ原材料内の異なるビット間の相互作用
　for k in range(j+1, num_bits_per_ingredient):
    weight_i_k = get_weight(i, k)
    bqm.add_interaction(q[i][j], q[i][k], 2 * F * weight_i_j * weight_i_k)


　同じ「成分」（iが同じ）内の異なるビット間（jとk）の相互作用をモデル化。

　重み (\text{weight}i][j]) と (\text{weight}i][k]) を基に、係数 (2 \cdot F \cdot \text{weight}i][j] \cdot \text{weight}i][k]) を設定。

###3.異なる原材料間の相互作用
　for m in range(i+1, num_ingredients):
    for n in range(num_bits_per_ingredient):
        weight_m_n = get_weight(m, n)
        bqm.add_interaction(q[i][j], q[m][n], 2 * F * weight_i_j * weight_m_n)

　異なる成分間（iとm）のビット（jとn）間の相互作用をモデル化。

　各成分間で全てのビットの組み合わせを考慮しており、係数は (2 \cdot F \cdot \text{weight}i][j] \cdot \text{weight}m][n])。

###4.線形項（-2の部分）
　bqm.add_variable(q[i][j], -2 * F * weight_i_j)

　線形項、つまり単純な一次関係を構築します。

　これはモデルのエネルギーを最適化するために「ペナルティ」を加える形です。係数は (-2 \cdot F \cdot \text{weight}i][j])。

## 全体の動き
　目的: 量子ビットに基づいてエネルギー関数を最小化すること。

　モデル構築手法:

　　1.自己ループやビット間の相互作用を設定。

　　2.総エネルギーが物理的に低くなる組み合わせを見つける。


___
---

## add_variable の解説
　バイナリ・クアドラティック・モデル（BQM）の構築において変数とその線形係数を追加する関数です。この機能は、モデルのエネルギー関数に含まれる線形項を定義するために使用されます。

###一般的な構造
　bqm.add_variable(variable, coefficient)
引数
　1.variable: モデル内の変数
　変数は通常、バイナリ値を取り、0または1になることが前提です。例: q[i][j] は特定のビットを表します。

　2.coefficient: 線形項の重み（定数）。
- この重みは、変数が取る値（0または1）に応じたエネルギーの影響を表します。
- 正の値であれば「ペナルティ」、負の値であれば「報酬」のような意味合いを持ちます。

###役割
　add_variable関数を使用することで、以下の点がモデルに反映されます:

　エネルギー関数の線形項を構築。

　各変数が単独でエネルギーに与える影響を設定。

　例えば、エネルギー関数の一部は次のように表せます： [ E = c_1 \cdot x_1 + c_2 \cdot x_2 + \ldots ] ここで、cは係数、xは変数です。

###具体例
　以下は、ある変数に線形項を追加する例です：

　variable_name = 'q0'
coefficient_value = -2.5
bqm.add_variable(variable_name, coefficient_value)

　これにより、変数q0が線形項の一部として追加され、エネルギー関数に (-2.5 \cdot q_0) が組み込まれます。

###応用
　add_variableと他の関数（例: add_interaction）を組み合わせることで、線形項だけでなく、変数間の相互作用項も構築できます。このようにしてBQMがエネルギー関数の全体を表現します。

***
---


##add_interactionの解説

　add_interactionは、バイナリ・クアドラティック・モデル（BQM）内で変数間の相互作用項を設定するための関数です。この関数は、エネルギー関数における二次項の係数を追加する役割を果たします。以下に、詳細な説明を提供します。

###一般的な構造
　bqm.add_interaction(variable_1, variable_2, coefficient)

引数
　1.variable_1: 最初の変数。

　　エネルギー関数における1つ目のバイナリ変数（例: ビット）。例：q[i][j]

　2.variable_2: 2番目の変数。

　　1つ目の変数と相互作用する対象となる変数。例：q[m][n]

　3.coefficient: 2つの変数間の相互作用係数。
- この係数は、2つの変数が同時に1である場合のエネルギーへの影響を決定します。

###役割
　add_interactionは、以下のような相互作用項をエネルギー関数に追加します： [ E = c_{12} \cdot x_1 \cdot x_2 ] ここで

　　(c_{12}): 係数（coefficient）。

　　(x_1, x_2): バイナリ変数。

　この項は、2つの変数が両方とも1である場合のエネルギーに寄与します。

###具体例
　以下は、相互作用を追加する例です：

　variable_1 = 'q0'
variable_2 = 'q1'
interaction_coefficient = 1.5

bqm.add_interaction(variable_1, variable_2, interaction_coefficient)

　このコードは、変数q0とq1が互いにエネルギー関数へ与える影響をモデル化します。具体的には、次のような項がエネルギー関数に追加されます： [ E = 1.5 \cdot q_0 \cdot q_1 ]

###応用
　add_interactionを使用することで、以下のようなケースをモデル化できます：

　　自己相互作用: 同じ変数のビット間の影響を考慮。

　　交差相互作用: 異なる変数やビット間の影響を考慮。

　例えば、量子アニーリングや組み合わせ最適化において、2つの変数間の関係性を最適化問題に反映できます。

###まとめ
　add_interactionは、相互作用係数を定義し、モデルのエネルギー関数を拡張するための重要なツールです。この関数を使うことで、変数間の関係を詳細に設定することができ、より精密なモデリングが可能になります。


---

---
栄養成分の目標値との差に対するペナルティを量子アニーリング（Binary Quadratic Model, BQM）を使用して設定する仕組みを構築しています。以下に各部分の解説を提供します。
---
##全体の目的
栄養成分（カロリー、タンパク質、脂質、炭水化物）に対して目標値との差を評価し、これらの差異に基づくペナルティをエネルギー関数に組み込む形です。
---
###コードの各部分の解説
1.外部ループ (for i in range(num_ingredients))
 - 各「原材料」について処理を行うループ。
 - **num_ingredients**は原材料の数を示します。

2.内部ループ (for j in range(num_bits_per_ingredient))
 - 各原材料の「ビット」ごとの処理を行います。
 - **num_bits_per_ingredient**は、1つの原材料に割り当てられたビット数。

3.カロリーのペナルティ

cal_contrib = A * ingredient_nutrition[f'ingredient{i+1}']['calories'] * weight
bqm.add_variable(q[i][j], 2 * cal_contrib * (-target_calories))

 - cal_contribは、各原材料（ingredient）についてカロリー寄与を計算しています。
  - ingredient_nutrition[f'ingredient{i+1}']['calories']: 現在の原材料のカロリー値。
  - weight: ビット（変数）の重み。
  - A: スケール係数。この係数はカロリーの影響を調整するために使用されます。

 - その後、bqm.add_variable()を使用してカロリー寄与に基づいた線形項を追加。- 2 * cal_contrib * (-target_calories):
  - target_calories: 目標とするカロリー値。
  - 計算結果にペナルティを加える形でエネルギー関数を修正します。
 - target_caloriesをかけることで、目標値との差異をペナルティ化。
 - **bqm.add_variable**により、各ビットの線形項が追加されます。

4.自己ループ（同一ビットの二次項）
if i == m and j == n:
    bqm.add_variable(q[i][j], cal_contrib * ingredient_nutrition[f'ingredient{i+1}']['calories'] * weight)

 - 同じ原材料内、同じビットに対する自己ループの項。
 - 二次項としてエネルギー関数に反映されます。



5.相互作用（異なるビット間のペナルティ）weight_m_n = get_weight(m, n)
cal_m_n = ingredient_nutrition[f'ingredient{m+1}']['calories'] * weight_m_n
bqm.add_interaction(q[i][j], q[m][n], 2 * cal_contrib * cal_m_n)

 - 異なる原材料や異なるビット間の相互作用を設定。
 - bqm.add_interactionを使用して二次項をエネルギー関数に追加。

6.タンパク質、脂質、炭水化物
 - カロリーと同様の方法で、それぞれの栄養素についてペナルティを追加します：
 - タンパク質:
prot_contrib = B * ingredient_nutrition[f'ingredient{i+1}']['protein'] * weight
bqm.add_variable(q[i][j], 2 * prot_contrib * (-target_protein))
 - 脂質:
fat_contrib = C * ingredient_nutrition[f'ingredient{i+1}']['fat'] * weight
bqm.add_variable(q[i][j], 2 * fat_contrib * (-target_fat))
 - 炭水化物:
carb_contrib = D * ingredient_nutrition[f'ingredient{i+1}']['carbs'] * weight
bqm.add_variable(q[i][j], 2 * carb_contrib * (-target_carbs))
---
モデル全体の流れ
- 線形項の設定: 各栄養素の目標値との差を反映。
- 二次項の設定:- 自己ループ項（同じ変数の二乗）。
- 相互作用項（異なる変数間の影響）。

---
---


このコードの部分についてさらに詳しく説明します。以下に、セクションごとに掘り下げて解説します。

1. カロリーのペナルティ追加
cal_contrib = A * ingredient_nutrition[f'ingredient{i+1}']['calories'] * weight
bqm.add_variable(q[i][j], 2 * cal_contrib * (-target_calories))


処理の流れ
- cal_contribは、各原材料（ingredient）についてカロリー寄与を計算しています。- ingredient_nutrition[f'ingredient{i+1}']['calories']: 現在の原材料のカロリー値。
- weight: ビット（変数）の重み。
- A: スケール係数。この係数はカロリーの影響を調整するために使用されます。

- その後、bqm.add_variable()を使用してカロリー寄与に基づいた線形項を追加。- 2 * cal_contrib * (-target_calories):- target_calories: 目標とするカロリー値。
- 計算結果にペナルティを加える形でエネルギー関数を修正します。




2. 自己ループの追加
if i == m and j == n:
    bqm.add_variable(q[i][j], cal_contrib * ingredient_nutrition[f'ingredient{i+1}']['calories'] * weight)


処理の流れ
- 自己ループは、同じビットが取るエネルギー寄与を考慮します。- 変数自身が二乗項となる影響を反映。
- ingredient_nutrition[f'ingredient{i+1}']['calories']:- 現在の原材料のカロリー値を取得。

- cal_contribはカロリーの影響度としてスケーリングされます。

- bqm.add_variable()により、変数自身への影響を線形項として追加。


3. 原材料間の相互作用
weight_m_n = get_weight(m, n)
cal_m_n = ingredient_nutrition[f'ingredient{m+1}']['calories'] * weight_m_n
bqm.add_interaction(q[i][j], q[m][n], 2 * cal_contrib * cal_m_n)


処理の流れ
- 異なる原材料間（iとm）や異なるビット間（jとn）の相互作用を構築します。
- get_weight(m, n)で、異なるビットの重みを取得。
- cal_m_n:- 異なる原材料（ingredient{m+1}）のカロリー寄与を計算。
- 計算に含まれる重み（weight_m_n）と栄養値を乗算。

- bqm.add_interaction()を使用して相互作用項をエネルギー関数に追加。- この部分は、2つの異なる変数（ビット）間の影響をモデル化しています。



4. その他の栄養素（タンパク質、脂質、炭水化物）
同様のロジックを使用して、タンパク質、脂質、炭水化物についてペナルティを構築しています。それぞれの部分は以下の通りです：
タンパク質
prot_contrib = B * ingredient_nutrition[f'ingredient{i+1}']['protein'] * weight
bqm.add_variable(q[i][j], 2 * prot_contrib * (-target_protein))


- タンパク質寄与の計算：B（スケール係数）を使用して重み付け。

脂質
fat_contrib = C * ingredient_nutrition[f'ingredient{i+1}']['fat'] * weight
bqm.add_variable(q[i][j], 2 * fat_contrib * (-target_fat))


- 脂質寄与の計算：C（スケール係数）を使用。

炭水化物
carb_contrib = D * ingredient_nutrition[f'ingredient{i+1}']['carbs'] * weight
bqm.add_variable(q[i][j], 2 * carb_contrib * (-target_carbs))


- 炭水化物寄与の計算：D（スケール係数）を使用。


総まとめ
このコードは、複数の栄養素（カロリー、タンパク質、脂質、炭水化物）について目標値との差を評価し、その差をペナルティとしてエネルギー関数に組み込む仕組みを構築しています。
- 線形項の追加 (add_variable):- 各ビットが単独で与える影響をモデル化。

- 二次項の追加 (add_interaction):- 原材料間やビット間の相互作用を設定。


---
---






In [393]:
# 元の定数項の追加部分 (コメントアウト)
# # 定数項の追加
# bqm.add_offset(A * target_calories**2 + B * target_protein**2 + C * target_fat**2 + D * target_carbs**2)
# --- 修正されたコスト関数項の追加 - 合計制約と栄養誤差 (統合) ---
# xi = sum(j=0 to 3) (2^j / 15.0) * q_i_j
# W_ij = (2^j / 15.0) -> xi = sum(j) W_ij * q_i_j

# H = H_合計制約 + H_栄養成分
# H_合計制約 = F * (sum(i) xi - 1)^2
# H_栄養成分 = sum(nut) Coeff_nut * (sum(i) NutVal_i_nut * xi - Target_nut)^2
# NutVal_i_nut は原材料iの「1gあたり」の栄養成分値
# Target_nut は「製品1gあたり」のターゲット値

# 各項を展開し、q_ij の一次項と二次項の係数を計算してBQMに追加します。

print("\n--- Block 3: Adding Constraint and Nutrition Error Terms (Corrected) ---") # 修正箇所：表示変更

# 線形項と二次項の係数を計算するための辞書を初期化
linear_biases = {var: 0.0 for var in bqm.variables}
quadratic_biases = {} # キーは (変数名1, 変数名2) のタプルで正規化 (辞書順)

# オフセットの合計
total_offset = 0.0

# 栄養成分のタイプ、ターゲット値、係数をリスト化 (修正箇所：追加)
nutrition_data = {
    'calories': {'target': target_calories, 'coeff': A},
    'protein' : {'target': target_protein,  'coeff': B},
    'fat'     : {'target': target_fat,      'coeff': C},
    'carbs'   : {'target': target_carbs,    'coeff': D},
}

# 全てのバイナリ変数ペア (q_i_j, q_k_l) についてループし、合算した係数を計算
for i in range(num_ingredients):
    for j in range(num_bits_per_ingredient):
        v_ij_name = q[i][j]
        w_ij = (2**j) / 15.0 # W_ij = (2^j / 15.0)

        # 各変数に対する線形バイアスを計算
        linear_bias_ij = 0.0

        # --- 合計制約からの線形項 (修正箇所：追加) ---
        # H_合計制約の展開から得られる q_ij の係数部分
        linear_bias_ij += F * (w_ij**2 - 2.0 * w_ij)

        # --- 栄養成分誤差からの線形項 (修正箇所：追加) ---
        # H_栄養成分の展開から得られる q_ij の係数部分の合計
        for nut_type, data in nutrition_data.items():
            target_val = data['target'] # 製品1gあたりのターゲット値
            coeff_nut  = data['coeff'] # A, B, C, D の係数
            nut_val_i_per_g = ingredient_nutrition_per_g[ingredient_keys[i]][nut_type] # 原材料iの1gあたりの栄養成分値 V_i_nut
            c_prime_ij_nut = nut_val_i_per_g * w_ij # C'_ij_nut = V_i_nut * W_ij

            linear_bias_ij += coeff_nut * (c_prime_ij_nut**2 - 2.0 * target_val * c_prime_ij_nut)

        # 計算した線形バイアスを一時辞書に格納
        linear_biases[v_ij_name] = linear_bias_ij

        # 各ペアに対する二次バイアスを計算
        for k in range(num_ingredients):
            for l in range(num_bits_per_ingredient):
                v_kl_name = q[k][l]
                # 重複を防ぎ、ペアを正規化 (変数名の辞書順を使用)
                if v_ij_name < v_kl_name:
                    w_kl = (2**l) / 15.0 # W_kl = (2^l / 15.0)

                    quadratic_bias_ijkl = 0.0

                    # --- 合計制約からの二次項 (修正箇所：追加) ---
                    # H_合計制約の展開から得られる q_ij * q_kl の係数 (展開で得られるのは F * W_ij * W_kl だが、BQMでは対称性考慮で2倍)
                    quadratic_bias_ijkl += F * 2.0 * w_ij * w_kl

                    # --- 栄養成分誤差からの二次項 (修正箇所：追加) ---
                    # H_栄養成分の展開から得られる q_ij * q_kl の係数部分の合計 (同様にBQMでは2倍)
                    # 係数は sum(nut) Coeff_nut * 2 * C'_ij_nut * C'_kl_nut
                    for nut_type, data in nutrition_data.items():
                        coeff_nut = data['coeff']
                        nut_val_i_per_g = ingredient_nutrition_per_g[ingredient_keys[i]][nut_type] # V_i_nut
                        nut_val_k_per_g = ingredient_nutrition_per_g[ingredient_keys[k]][nut_type] # V_k_nut
                        c_prime_ij_nut = nut_val_i_per_g * w_ij # C'_ij_nut
                        c_prime_kl_nut = nut_val_k_per_g * w_kl # C'_kl_nut

                        quadratic_bias_ijkl += coeff_nut * 2.0 * c_prime_ij_nut * c_prime_kl_nut

                    # 計算した二次バイアスを一時辞書に格納
                    pair = (v_ij_name, v_kl_name)
                    quadratic_biases[pair] = quadratic_bias_ijkl


# --- 定数オフセットを計算 (修正箇所：追加) ---
# 合計制約からの定数項は F * (-1)^2 = F
total_offset += F
# 栄養成分誤差からの定数項は sum(nut) Coeff_nut * (-Target_nut)^2 = sum(nut) Coeff_nut * Target_nut^2
for nut_type, data in nutrition_data.items():
    target_val = data['target']
    coeff_nut = data['coeff']
    total_offset += coeff_nut * (target_val**2)


# --- 計算した係数をBQMにまとめて追加 (修正箇所：追加) ---
# ここで初めてBQMオブジェクトに計算結果を反映させる
for var_name, bias in linear_biases.items():
    bqm.add_linear(var_name, bias)

for (v1_name, v2_name), bias in quadratic_biases.items():
     bqm.add_quadratic(v1_name, v2_name, bias)

bqm.add_offset(total_offset)

print("  - All terms added to BQM.")


--- Block 3: Adding Constraint and Nutrition Error Terms (Corrected) ---
  - All terms added to BQM.


<ipython-input-393-aea0fe99fab3>:108: DeprecationWarning: BinaryQuadraticModel.add_offset(bias) is deprecated since dimod 0.10.0 and will be removed in 0.12.0. Please use bqm.offset += bias.
  bqm.add_offset(total_offset)


In [394]:
# Block 4: 原材料順序の制約 (H_順序制約) - この実装では含めない (前回の修正済みと同じ)

# H_順序制約 = E * sum(i=0 to num_ingredients-2) max(0, x_{i+1} - x_i)
# この項は、このスクリプトではBQMに含めません。
# 実際の応用では、この制約の重要性に応じて、実装方法を検討する必要があります。

print("\n--- Block 4: Order Constraint Term (Not Included in this implementation) ---")
print("  - The order constraint H_順序制約 is not included as its QUBO conversion is complex.")
print("  - Requires techniques like auxiliary variables if needed.")


--- Block 4: Order Constraint Term (Not Included in this implementation) ---
  - The order constraint H_順序制約 is not included as its QUBO conversion is complex.
  - Requires techniques like auxiliary variables if needed.


In [395]:
# サンプラーを使用してQUBOを解く
sampler = SASampler()
sampleset = sampler.sample(bqm, num_reads=100)

In [396]:
# prompt: samplesetの１番目の値のname抜きで表示

#print(list(sampleset.first.sample.values())[0]) # ok
#print(list(sampleset.first.sample.values())) # ok
#print(list(sampleset.first.sample.keys())) # ok
#print(list(sampleset.first.sample.items())) # ok
print(list(sampleset.first.sample.items())) # ok


[('q_0_0', np.int8(0)), ('q_0_1', np.int8(1)), ('q_0_10', np.int8(0)), ('q_0_11', np.int8(0)), ('q_0_12', np.int8(0)), ('q_0_13', np.int8(0)), ('q_0_14', np.int8(0)), ('q_0_15', np.int8(0)), ('q_0_2', np.int8(1)), ('q_0_3', np.int8(0)), ('q_0_4', np.int8(0)), ('q_0_5', np.int8(0)), ('q_0_6', np.int8(0)), ('q_0_7', np.int8(0)), ('q_0_8', np.int8(0)), ('q_0_9', np.int8(0)), ('q_1_0', np.int8(0)), ('q_1_1', np.int8(1)), ('q_1_10', np.int8(0)), ('q_1_11', np.int8(0)), ('q_1_12', np.int8(0)), ('q_1_13', np.int8(0)), ('q_1_14', np.int8(0)), ('q_1_15', np.int8(0)), ('q_1_2', np.int8(1)), ('q_1_3', np.int8(0)), ('q_1_4', np.int8(0)), ('q_1_5', np.int8(0)), ('q_1_6', np.int8(0)), ('q_1_7', np.int8(0)), ('q_1_8', np.int8(0)), ('q_1_9', np.int8(0)), ('q_2_0', np.int8(1)), ('q_2_1', np.int8(1)), ('q_2_10', np.int8(0)), ('q_2_11', np.int8(0)), ('q_2_12', np.int8(0)), ('q_2_13', np.int8(0)), ('q_2_14', np.int8(0)), ('q_2_15', np.int8(0)), ('q_2_2', np.int8(0)), ('q_2_3', np.int8(0)), ('q_2_4', np.in

In [397]:
sampleset

SampleSet(rec.array([([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 0.5895977 , 1),
           ([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.0159874 , 1),
           ([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 0.08112335, 1),
           ([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], 0.64660072, 1),
           ([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 0.25404739, 1),
           ([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,

In [398]:
# prompt: samplesetの１番目の要素を表示

sampleset.first


Sample(sample={'q_0_0': np.int8(0), 'q_0_1': np.int8(1), 'q_0_10': np.int8(0), 'q_0_11': np.int8(0), 'q_0_12': np.int8(0), 'q_0_13': np.int8(0), 'q_0_14': np.int8(0), 'q_0_15': np.int8(0), 'q_0_2': np.int8(1), 'q_0_3': np.int8(0), 'q_0_4': np.int8(0), 'q_0_5': np.int8(0), 'q_0_6': np.int8(0), 'q_0_7': np.int8(0), 'q_0_8': np.int8(0), 'q_0_9': np.int8(0), 'q_1_0': np.int8(0), 'q_1_1': np.int8(1), 'q_1_10': np.int8(0), 'q_1_11': np.int8(0), 'q_1_12': np.int8(0), 'q_1_13': np.int8(0), 'q_1_14': np.int8(0), 'q_1_15': np.int8(0), 'q_1_2': np.int8(1), 'q_1_3': np.int8(0), 'q_1_4': np.int8(0), 'q_1_5': np.int8(0), 'q_1_6': np.int8(0), 'q_1_7': np.int8(0), 'q_1_8': np.int8(0), 'q_1_9': np.int8(0), 'q_2_0': np.int8(1), 'q_2_1': np.int8(1), 'q_2_10': np.int8(0), 'q_2_11': np.int8(0), 'q_2_12': np.int8(0), 'q_2_13': np.int8(0), 'q_2_14': np.int8(0), 'q_2_15': np.int8(0), 'q_2_2': np.int8(0), 'q_2_3': np.int8(0), 'q_2_4': np.int8(0), 'q_2_5': np.int8(0), 'q_2_6': np.int8(0), 'q_2_7': np.int8(0), '

In [399]:
# reshape を試みる誤った行はコメントアウト
print(sampleset.first.sample)
print(sampleset.first.sample)
# print(sampleset.record[1][0].reshape(3,4))
# print(sampleset.record[0][0].reshape(4,3))
# print(sampleset.record[1][0].reshape(4,3))

{'q_0_0': np.int8(0), 'q_0_1': np.int8(1), 'q_0_10': np.int8(0), 'q_0_11': np.int8(0), 'q_0_12': np.int8(0), 'q_0_13': np.int8(0), 'q_0_14': np.int8(0), 'q_0_15': np.int8(0), 'q_0_2': np.int8(1), 'q_0_3': np.int8(0), 'q_0_4': np.int8(0), 'q_0_5': np.int8(0), 'q_0_6': np.int8(0), 'q_0_7': np.int8(0), 'q_0_8': np.int8(0), 'q_0_9': np.int8(0), 'q_1_0': np.int8(0), 'q_1_1': np.int8(1), 'q_1_10': np.int8(0), 'q_1_11': np.int8(0), 'q_1_12': np.int8(0), 'q_1_13': np.int8(0), 'q_1_14': np.int8(0), 'q_1_15': np.int8(0), 'q_1_2': np.int8(1), 'q_1_3': np.int8(0), 'q_1_4': np.int8(0), 'q_1_5': np.int8(0), 'q_1_6': np.int8(0), 'q_1_7': np.int8(0), 'q_1_8': np.int8(0), 'q_1_9': np.int8(0), 'q_2_0': np.int8(1), 'q_2_1': np.int8(1), 'q_2_10': np.int8(0), 'q_2_11': np.int8(0), 'q_2_12': np.int8(0), 'q_2_13': np.int8(0), 'q_2_14': np.int8(0), 'q_2_15': np.int8(0), 'q_2_2': np.int8(0), 'q_2_3': np.int8(0), 'q_2_4': np.int8(0), 'q_2_5': np.int8(0), 'q_2_6': np.int8(0), 'q_2_7': np.int8(0), 'q_2_8': np.int

In [400]:
# 最良の解を取得
best_sample = sampleset.first.sample
# エネルギーも取得 (修正箇所：追加)
best_energy = sampleset.first.energy

print(f"\n--- Block 5: Solving the BQM with {bqm.num_variables} variables ---") # 修正箇所：表示変更
print("  - Solving complete.") # 修正箇所：表示変更
print(f"  - Best energy found: {best_energy}") # 修正箇所：追加
print(f"  - Best sample: {best_sample}") # 修正箇所：追加



--- Block 5: Solving the BQM with 48 variables ---
  - Solving complete.
  - Best energy found: 0.013776779174804688
  - Best sample: {'q_0_0': np.int8(0), 'q_0_1': np.int8(1), 'q_0_10': np.int8(0), 'q_0_11': np.int8(0), 'q_0_12': np.int8(0), 'q_0_13': np.int8(0), 'q_0_14': np.int8(0), 'q_0_15': np.int8(0), 'q_0_2': np.int8(1), 'q_0_3': np.int8(0), 'q_0_4': np.int8(0), 'q_0_5': np.int8(0), 'q_0_6': np.int8(0), 'q_0_7': np.int8(0), 'q_0_8': np.int8(0), 'q_0_9': np.int8(0), 'q_1_0': np.int8(0), 'q_1_1': np.int8(1), 'q_1_10': np.int8(0), 'q_1_11': np.int8(0), 'q_1_12': np.int8(0), 'q_1_13': np.int8(0), 'q_1_14': np.int8(0), 'q_1_15': np.int8(0), 'q_1_2': np.int8(1), 'q_1_3': np.int8(0), 'q_1_4': np.int8(0), 'q_1_5': np.int8(0), 'q_1_6': np.int8(0), 'q_1_7': np.int8(0), 'q_1_8': np.int8(0), 'q_1_9': np.int8(0), 'q_2_0': np.int8(1), 'q_2_1': np.int8(1), 'q_2_10': np.int8(0), 'q_2_11': np.int8(0), 'q_2_12': np.int8(0), 'q_2_13': np.int8(0), 'q_2_14': np.int8(0), 'q_2_15': np.int8(0), 'q_2_2

In [401]:
# 結果の表示
#print("推定された配合比率:")
#for ingredient, ratio in estimated_x.items():
#    name = ingredient_nutrition[ingredient]['name']
#    print(f"{ingredient} ({name}): {ratio:.4f}")
print("\n--- Block 6: Analyzing Results ---") # 修正箇所：表示変更

# 最良の解 (バイナリ変数 q_i_j の値) を取得
# best_sample は {変数名: 0 or 1} の辞書

# 各原材料の比率 x_i を計算 (修正箇所：移動)
estimated_x = {}
x = np.zeros(num_ingredients)
for i in range(num_ingredients):
    for j in range(num_bits_per_ingredient):
        var_name = q[i][j]
        # バイナリ変数の値を取得し、係数を掛けて加算
        x[i] += float(best_sample[var_name]) * (2**j) / 15.0  # 2025/04/28 8bit化で、integerからfloatへ
    estimated_x[ingredient_keys[i]] = x[i] # 辞書にも格納 (修正箇所：キーをingredient_keysから取得)


print("\nCalculated ingredient proportions (x_i):") # 修正箇所：表示変更
for i in range(num_ingredients): # 修正箇所：ループ方法変更
    print(f"  {ingredient_keys[i]} ({ingredient_nutrition_per_g[ingredient_keys[i]]['name']}): {x[i]:.4f}")


# 合計比率を確認 (修正箇所：変数名変更)
total_proportion = np.sum(x)
print(f"\nTotal proportion (sum of x_i): {total_proportion:.4f}")


--- Block 6: Analyzing Results ---

Calculated ingredient proportions (x_i):
  ingredient1 (きなこ（大豆を含む）（国内製造）): 0.4000
  ingredient2 (砂糖): 0.4000
  ingredient3 (水飴): 0.2000

Total proportion (sum of x_i): 1.0000


In [402]:
num_bits_per_ingredient

16

In [403]:
# 元の栄養成分計算部分 (コメントアウト)

# 計算された栄養成分値 (製品1gあたりとして計算) (修正箇所：追加)
calculated_nut_per_g = {}
nutrition_types_list = ['calories', 'protein', 'fat', 'carbs'] # 修正箇所：リストを定義
for nut_type in nutrition_types_list:
     calculated_nut_per_g[nut_type] = sum(x[i] * ingredient_nutrition_per_g[ingredient_keys[i]][nut_type] for i in range(num_ingredients))

print("\nCalculated Nutritional Values (per gram of product):") # 修正箇所：表示変更
nutrition_data_print = {
    'calories': {'target': target_calories, 'coeff': A},
    'protein' : {'target': target_protein, 'coeff': B},
    'fat'     : {'target': target_fat, 'coeff': C},
    'carbs'   : {'target': target_carbs, 'coeff': D},
} # 修正箇所：表示用データ定義
for nut_type, calc_val in calculated_nut_per_g.items():
    target_val = nutrition_data_print[nut_type]['target']
    print(f"  {nut_type.capitalize()}: {calc_val:.4f} (Target per gram: {target_val:.4f})")


# 計算された栄養成分値 (製品75gあたりに換算して目標値と比較) (修正箇所：追加)
print(f"\nCalculated Nutritional Values ({data_weight}g total):")
# 元の75gターゲット値を取得するための辞書または方法 (修正箇所：追加)
target_75g_values = {
    'calories': target_calories_75g,
    'protein' : target_protein_75g,
    'fat'     : target_fat_75g,
    'carbs'   : target_carbs_75g,
}
for nut_type, calc_val_per_g in calculated_nut_per_g.items():
    calc_val_75g = calc_val_per_g * data_weight
    target_val_75g = target_75g_values[nut_type]
    print(f"  {nut_type.capitalize()}: {calc_val_75g:.2f} (Target {data_weight}g: {target_val_75g:.2f})")




Calculated Nutritional Values (per gram of product):
  Calories: 0.9502 (Target per gram: 1.0000)
  Protein: 0.9175 (Target per gram: 1.0000)
  Fat: 1.0419 (Target per gram: 1.0000)
  Carbs: 1.0523 (Target per gram: 1.0000)

Calculated Nutritional Values (75.0g total):
  Calories: 71.26 (Target 75.0g: 299.00)
  Protein: 68.81 (Target 75.0g: 12.00)
  Fat: 78.14 (Target 75.0g: 7.40)
  Carbs: 78.92 (Target 75.0g: 46.40)


In [404]:
# 各コスト項の値を計算して確認 (オプション) (修正箇所：追加)
# 合計制約ペナルティ
sum_constraint_penalty = F * (total_proportion - 1.0)**2
print(f"\nSum Constraint Penalty: {sum_constraint_penalty:.4f}")

# 栄養誤差ペナルティ (製品1gあたりで計算した誤差を使用)
nut_error_penalty = 0.0
# nutrition_data_print は Block 6 の上で定義済み
for nut_type, calc_val_per_g in calculated_nut_per_g.items():
    target_val_per_g = nutrition_data_print[nut_type]['target']
    coeff_nut = nutrition_data_print[nut_type]['coeff']
    nut_error_penalty += coeff_nut * (calc_val_per_g - target_val_per_g)**2

print(f"Nutrition Error Penalty (based on per gram error): {nut_error_penalty:.4f}")



Sum Constraint Penalty: 0.0000
Nutrition Error Penalty (based on per gram error): 0.0138


In [405]:
# BQMのエネルギーはこれらの項の合計とオフセットになります。
# BQMのオフセットは Block 3 で計算した total_offset
# total_offset は Block 3 で計算済み
print(f"Total Calculated Cost (excluding order constraint, sum of penalties + offset): {sum_constraint_penalty + nut_error_penalty + total_offset:.4f}")
print(f"Best Energy from Solver:  {best_energy:.4f}") # これらが近い値になるはず


Total Calculated Cost (excluding order constraint, sum of penalties + offset): 104.0138
Best Energy from Solver:  0.0138


# 結果の集約表示

In [406]:
# 結果の集約表示
print("bit数=", num_bits_per_ingredient)
#print(f"\n")

for nut_type, calc_val in calculated_nut_per_g.items():
    target_val = nutrition_data_print[nut_type]['target']
    print(f"  {nut_type.capitalize()}: {calc_val:.4f} (Target per gram: {target_val:.4f})")
#print(f"\n")

print(f"\nSum Constraint Penalty: {sum_constraint_penalty:.4f}")
print(f"Nutrition Error Penalty (based on per gram error): {nut_error_penalty:.4f}")
#print(f"\n")

print(f"  - Best energy found: {best_energy}") # 修正箇所：追加
print(f"  - Best sample: {best_sample}") # 修正箇所：追加

bit数= 16
  Calories: 0.9502 (Target per gram: 1.0000)
  Protein: 0.9175 (Target per gram: 1.0000)
  Fat: 1.0419 (Target per gram: 1.0000)
  Carbs: 1.0523 (Target per gram: 1.0000)

Sum Constraint Penalty: 0.0000
Nutrition Error Penalty (based on per gram error): 0.0138
  - Best energy found: 0.013776779174804688
  - Best sample: {'q_0_0': np.int8(0), 'q_0_1': np.int8(1), 'q_0_10': np.int8(0), 'q_0_11': np.int8(0), 'q_0_12': np.int8(0), 'q_0_13': np.int8(0), 'q_0_14': np.int8(0), 'q_0_15': np.int8(0), 'q_0_2': np.int8(1), 'q_0_3': np.int8(0), 'q_0_4': np.int8(0), 'q_0_5': np.int8(0), 'q_0_6': np.int8(0), 'q_0_7': np.int8(0), 'q_0_8': np.int8(0), 'q_0_9': np.int8(0), 'q_1_0': np.int8(0), 'q_1_1': np.int8(1), 'q_1_10': np.int8(0), 'q_1_11': np.int8(0), 'q_1_12': np.int8(0), 'q_1_13': np.int8(0), 'q_1_14': np.int8(0), 'q_1_15': np.int8(0), 'q_1_2': np.int8(1), 'q_1_3': np.int8(0), 'q_1_4': np.int8(0), 'q_1_5': np.int8(0), 'q_1_6': np.int8(0), 'q_1_7': np.int8(0), 'q_1_8': np.int8(0), 'q_1_

### 4. コスト関数のQUBO形式への変換

### 5. 総合目的関数 (QUBOの構築 - 困難な部分を含む)
H = H_栄養成分 + H_順序制約 + F(Σ x_i - 1)^2  
上記の H_nutrition, H_sum は x_i の式（q変数の組み合わせ）で表現されているため、
BQMにこれらの項を二次形式として追加していく必要があります。
しかし、H_order の max関数は直接的な二次形式への変換が難しいため、
厳密なQUBOとしては構築が複雑になります。


### 6. SimulatedAnnealingSampler の利用

### 7. 結果の解釈